<a href="https://colab.research.google.com/github/sidesh27/Flight-Delay-Prediction/blob/master/Pipelined_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import seaborn as sns
import math
from sklearn.svm import LinearSVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor
from sklearn.utils.validation import column_or_1d
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import ADASYN
from sklearn.pipeline import Pipeline

In [ ]:
df = pd.read_csv("drive/My Drive/temp_merged.csv")

In [ ]:
c_y = df['ArrDel15']
X = df.drop(columns=['Unnamed: 0', 'FlightDate', 'Quarter', 'Year', 'Month', 'DayofMonth',
       'Origin', 'Dest','ArrTime','ArrDel15',])

In [ ]:
cx_train, cx_test, cy_train, cy_test = train_test_split(X, c_y, train_size=0.7,random_state = 0)

In [ ]:
cx_train = cx_train.drop(columns=['ArrDelayMinutes']) 

test_delay = cx_test['ArrDelayMinutes']

cx_test = cx_test.drop(columns=['ArrDelayMinutes']) 

##CLASSIFICATION - XGBOOST CLASSSIFIER

In [ ]:
xg = XGBClassifier()
clf = xg.fit(cx_train, cy_train)
cy_pred = xg.predict(cx_test)

In [ ]:
dic={'test_del':test_delay , 'cy_pred':list(cy_pred)}
new_df = pd.DataFrame(dic)
new_df = pd.concat([new_df,cx_test],axis=1)
new_df = new_df[new_df.cy_pred == 1]

TEST SET FOR PIPELINING FROM CLASSIFICATION TO REGRESSION

In [ ]:

ry_test = new_df['test_del']
rx_test = new_df.drop(columns=['cy_pred','test_del'])

##REGRESSION MODEL - XGBOOST REGRESSOR 

In [ ]:
df = pd.read_csv("drive/My Drive/temp_merged.csv")

In [ ]:
df = df[df.ArrDel15==1]

In [ ]:
y = df['ArrDelayMinutes']
X = df.drop(columns=['Unnamed: 0', 'FlightDate', 'Quarter', 'Year', 'Month', 'DayofMonth',
       'Origin', 'Dest','ArrTime','ArrDel15','ArrDelayMinutes'])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.7,random_state = 0)

In [ ]:
xgr = XGBRegressor()
reg = xgr.fit(x_train,y_train)
y_pred = xgr.predict(rx_test)

In [ ]:
print("MAE : ",mean_absolute_error(ry_test, y_pred))
print("RMSE : ",math.sqrt(mean_squared_error(ry_test, y_pred)))
print("R2_SCORE : ",r2_score(ry_test,y_pred))

MAE :  13.413472129067241
RMSE :  18.323108197962092
R2_SCORE :  0.9485916327918534


##REGRESSION ANALYSIS - PIPELINED MODEL

In [ ]:
 def reg_analysis(a,b): 
  rx_test1 = new_df[(new_df.test_del>=a) & (new_df.test_del<=b) ]
  ry_test1 = rx_test1['test_del']
  rx_test1 = rx_test1.drop(columns=['cy_pred','test_del'])
  y_pred = xgr.predict(rx_test1)
  print("MAE : ",mean_absolute_error(ry_test1, y_pred))
  print("RMSE : ",math.sqrt(mean_squared_error(ry_test1, y_pred)))
  print("R2_SCORE : ",r2_score(ry_test1,y_pred))

In [ ]:
reg_analysis(0.0,100.0)

MAE :  12.410241266818463
RMSE :  15.723376450535051
R2_SCORE :  0.6151795202907224


In [ ]:
reg_analysis(100.0,200.0)

MAE :  16.57566849568301
RMSE :  24.306976678492873
R2_SCORE :  0.20939498694162617


In [ ]:
reg_analysis(200.0,500.0)

MAE :  18.96755177952419
RMSE :  30.009444854243657
R2_SCORE :  0.798924399620389


In [ ]:
reg_analysis(500.0,1000.0)

MAE :  17.1071903684744
RMSE :  27.09615800892327
R2_SCORE :  0.9647435341642441


In [ ]:
reg_analysis(1000.0,2000.0)